In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import RobustScaler

import optuna
import lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import json

plt.rc('font', family='NanumBarunGothic')

In [31]:
train_df = pd.read_csv(r"C:\Users\User\LG_Aimers\MainTask\train.csv")
train_id = train_df.pop('ID')
test_df = pd.read_csv(r"C:\Users\User\LG_Aimers\MainTask\test.csv")
test_id = test_df.pop('ID')

In [32]:
categorical_columns = [
    "시술 시기 코드",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "난자 출처",
    "정자 출처",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [33]:
numeric_columns = [
    "이식된 배아 수",
    "시술 당시 나이",
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 기증자 나이",
    "정자 기증자 나이"
]

In [34]:
times_values = ["총 시술 횟수",
                "클리닉 내 총 시술 횟수",
                "IVF 시술 횟수",
                "DI 시술 횟수",
                "총 임신 횟수",
                "IVF 임신 횟수",
                "DI 임신 횟수",
                "총 출산 횟수",
                "IVF 출산 횟수",
                "DI 출산 횟수"]

donar_age_values = ["난자 기증자 나이", "정자 기증자 나이"]

train_df["시술 당시 나이"] = train_df["시술 당시 나이"].map({
    "만18-34세" : 1,
    "만35-37세" : 2,
    "만38-39세" : 3,
    "만40-42세" : 4,
    "만43-44세" : 5,
    "만45-50세" : 6,
    "알 수 없음" : 0
})

train_df[times_values] = train_df[times_values].map(lambda x : int(x[0]))

for donar_age_value in donar_age_values:
    train_df[donar_age_value] = train_df[donar_age_value].map({
        '만20세 이하' : 1,
        '만21-25세' : 2,
        '만26-30세' : 3,
        '만31-35세' : 4,
        '만36-40세' : 5,
        '만41-45세' : 6,
        "알 수 없음" : 0
    })

test_df["시술 당시 나이"] = test_df["시술 당시 나이"].map({
    "만18-34세" : 1,
    "만35-37세" : 2,
    "만38-39세" : 3,
    "만40-42세" : 4,
    "만43-44세" : 5,
    "만45-50세" : 6,
    "알 수 없음" : 0
})

test_df[times_values] = test_df[times_values].map(lambda x : int(x[0]))

for donar_age_value in donar_age_values:
    test_df[donar_age_value] = test_df[donar_age_value].map({
        '만20세 이하' : 1,
        '만21-25세' : 2,
        '만26-30세' : 3,
        '만31-35세' : 4,
        '만36-40세' : 5,
        '만41-45세' : 6,
        "알 수 없음" : 0
    })

In [35]:
# Fill missing values in numeric columns using the mean
train_df[numeric_columns] = train_df[numeric_columns].fillna(train_df[numeric_columns].median())
test_df[numeric_columns] = test_df[numeric_columns].fillna(train_df[numeric_columns].median())

train_df = train_df.fillna("NA")
test_df = test_df.fillna("NA")

In [36]:
if train_df.isnull().any().any():
    print(train_df.isnull().sum())
if test_df.isnull().any().any():
    print(test_df.isnull().sum())

In [37]:
# Convert categorical columns to category dtype
for col in categorical_columns:
    if col in train_df.columns:
        train_df[col] = train_df[col].astype("category")
        test_df[col] = test_df[col].astype("category")
    else:
        categorical_columns.remove(col)

In [38]:
# Function to safely divide and replace division by zero with 99
def safe_divide(numerator, denominator):
    return numerator / (denominator + 1e-6)

# Apply the safe division function to each calculation
train_df["미세주입에서 생성된 배아 비율1"] = safe_divide(train_df["미세주입에서 생성된 배아 수"], train_df["총 생성 배아 수"])
test_df["미세주입에서 생성된 배아 비율1"] = safe_divide(test_df["미세주입에서 생성된 배아 수"], test_df["총 생성 배아 수"])

train_df["저장된 배아 비율"] = safe_divide(train_df["저장된 배아 수"], train_df["총 생성 배아 수"])
test_df["저장된 배아 비율"] = safe_divide(test_df["저장된 배아 수"], test_df["총 생성 배아 수"])

train_df["미세주입 후 저장된 배아 비율"] = safe_divide(train_df["미세주입 후 저장된 배아 수"], train_df["저장된 배아 수"])
test_df["미세주입 후 저장된 배아 비율"] = safe_divide(test_df["미세주입 후 저장된 배아 수"], test_df["저장된 배아 수"])

train_df["혼합된 난자 비율"] = safe_divide(train_df["혼합된 난자 수"], train_df["수집된 신선 난자 수"] + train_df["해동 난자 수"])
test_df["혼합된 난자 비율"] = safe_divide(test_df["혼합된 난자 수"], test_df["수집된 신선 난자 수"] + test_df["해동 난자 수"])

train_df["저장된 신선 난자 비율"] = safe_divide(train_df["저장된 신선 난자 수"], train_df["수집된 신선 난자 수"])
test_df["저장된 신선 난자 비율"] = safe_divide(test_df["저장된 신선 난자 수"], test_df["수집된 신선 난자 수"])

train_df["미세주입된 난자 비율"] = safe_divide(train_df["미세주입된 난자 수"], train_df["혼합된 난자 수"])
test_df["미세주입된 난자 비율"] = safe_divide(test_df["미세주입된 난자 수"], test_df["혼합된 난자 수"])


In [39]:
# Apply to multiple numerical columns
scaler = RobustScaler()
train_df[numeric_columns] = scaler.fit_transform(train_df[numeric_columns])
test_df[numeric_columns] = scaler.transform(test_df[numeric_columns])

# 학습

# LIGHTGBM

In [47]:
# #X_train, X_val, y_train, y_val = train_test_split(train_df.drop(columns=['임신 성공 여부']), train_df['임신 성공 여부'], test_size=0.2, random_state=42, stratify=train_df['임신 성공 여부'])

def objective(trial):
    # Convert float categorical columns to int (if necessary)
    for col in categorical_columns:
        if train_df[col].dtype == 'float64':
            train_df[col] = train_df[col].astype(int)

    # Prepare dataset
    X = train_df.drop(columns=['임신 성공 여부'])
    y = train_df['임신 성공 여부']
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Define hyperparameter search space
    params = {
        "objective": "binary",  # Binary classification
        "metric": "auc",  # Optimize AUC
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 20),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 3.0),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 10.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
        "random_state": 42,
        "verbose": -1,
    }

    # Create LightGBM datasets
    dtrain = lightgbm.Dataset(train_x, label=train_y, categorical_feature=categorical_columns, free_raw_data=False)
    dvalid = lightgbm.Dataset(valid_x, label=valid_y, reference=dtrain, categorical_feature=categorical_columns, free_raw_data=False)

    # Train the model
    model = lightgbm.train(
        params,
        dtrain,
        valid_sets=[dvalid],
        valid_names=["valid"],
        num_boost_round=2000
    )

    # Predict probabilities for the positive class (1)
    y_pred_proba = model.predict(valid_x, num_iteration=model.best_iteration)

    # Calculate ROC-AUC score
    roc_auc = roc_auc_score(valid_y, y_pred_proba)

    return roc_auc

# Create a study and optimize
study = optuna.create_study(direction="maximize")  # Maximize ROC-AUC
study.optimize(objective, n_trials=1000)

# Display the best hyperparameters
print("Best Hyperparameters:")
print(study.best_params)

# # Save best parameters
# with open('/content/drive/MyDrive/LGAimers6/Models/LightGBM/lightgbm_params.json', 'w') as f:
#     json.dump(study.best_params, f)

[I 2025-02-19 14:26:22,435] A new study created in memory with name: no-name-653497c0-f5e5-4c18-bbc9-fd9dcf8d887c
C:\Users\User\anaconda3\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2025-02-19 14:26:34,968] Trial 0 finished with value: 0.6998501546073528 and parameters: {'n_estimators': 1293, 'learning_rate': 0.24893268424687845, 'num_leaves': 131, 'max_depth': 8, 'min_child_samples': 10, 'min_child_weight': 2.1829720977393117, 'feature_fraction': 0.8963182971827544, 'bagging_fraction': 0.8503536242536496, 'bagging_freq': 9, 'lambda_l1': 0.49211244193239745, 'lambda_l2': 9.800672401440993}. Best is trial 0 with value: 0.6998501546073528.
C:\Users\User\anaconda3\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it

Best Hyperparameters:
{'n_estimators': 1023, 'learning_rate': 0.02314996002308199, 'num_leaves': 119, 'max_depth': 4, 'min_child_samples': 15, 'min_child_weight': 0.48437772296665066, 'feature_fraction': 0.704521941506332, 'bagging_fraction': 0.8870490339666385, 'bagging_freq': 5, 'lambda_l1': 8.290783912994241, 'lambda_l2': 7.740417086782628}


In [53]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score

X = train_df.drop(columns=['임신 성공 여부'])
y = train_df['임신 성공 여부']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'n_estimators': 1023,
    'learning_rate': 0.02314996002308199,
    'num_leaves': 119,
    'max_depth': 4,
    'min_child_samples': 15,
    'min_child_weight': 0.48437772296665066,
    'feature_fraction': 0.704521941506332,
    'bagging_fraction': 0.8870490339666385,
    'bagging_freq': 5,
    'lambda_l1': 8.290783912994241,
    'lambda_l2': 7.740417086782628,
    'objective': 'binary',  
    'metric': 'auc',  
    'verbose': -1
}

model = lgb.train(params,
                  train_data,
                  valid_sets=[val_data],
                  num_boost_round=params['n_estimators'],
                  )

y_pred_proba = model.predict(X_val, num_iteration=model.best_iteration)

roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC: {roc_auc}')


C:\Users\User\anaconda3\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


ROC-AUC: 0.7397832241013509


In [55]:
test_proba = model.predict(test_df, num_iteration=model.best_iteration)

In [59]:
test_proba = pd.DataFrame(test_proba)
test_proba.to_csv('submission.csv',encoding='utf-8-sig')

# CATBOOST

In [685]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

X = train_df.drop(['임신 성공 여부'],axis=1)
y = train_df["임신 성공 여부"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

train_pool = Pool(X_train, y_train, cat_features=categorical_columns)  # 범주형 변수 인덱스 지정
test_pool = Pool(X_test, y_test, cat_features=categorical_columns)

model = CatBoostClassifier(
    iterations=1000,      
    depth=6,              
    learning_rate=0.05,   
    loss_function='Logloss', 
    eval_metric='AUC',   
    class_weights=[1, 3], 
    random_seed=42,
    verbose=100          
)

model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50, use_best_model=True)

y_pred_prob = model.predict_proba(X_test)[:, 1]  
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f"Test ROC-AUC Score: {roc_auc:.4f}")


0:	test: 0.7157363	best: 0.7157363 (0)	total: 190ms	remaining: 3m 10s
100:	test: 0.7347535	best: 0.7347535 (100)	total: 18.2s	remaining: 2m 42s
200:	test: 0.7362088	best: 0.7362088 (200)	total: 38.9s	remaining: 2m 34s


KeyboardInterrupt: 

# CATBOOST X LGBM


In [174]:
cat_features = categorical_columns

X = train_df.drop(['임신 성공 여부'],axis=1)
y = train_df["임신 성공 여부"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

cat_model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    loss_function='Logloss',
    eval_metric='AUC',
    class_weights=[1, 3],
    random_seed=42,
    verbose=100
)

lgb_model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    boosting_type='gbdt',
    objective='binary',
    metric='auc',
    class_weight={0: 1, 1: 3},
    random_state=42
)

cat_model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50, use_best_model=True)

X_train[categorical_columns] = X_train[categorical_columns].astype('category')
X_test[categorical_columns] = X_test[categorical_columns].astype('category')

lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

cat_pred_prob = cat_model.predict_proba(X_test)[:, 1]
lgb_pred_prob = lgb_model.predict_proba(X_test)[:, 1]

ensemble_pred_prob = (cat_pred_prob + lgb_pred_prob) / 2

roc_auc = roc_auc_score(y_test, ensemble_pred_prob)
print(f"Ensemble Test ROC-AUC Score: {roc_auc:.4f}")

0:	test: 0.7157363	best: 0.7157363 (0)	total: 168ms	remaining: 2m 47s
100:	test: 0.7347535	best: 0.7347535 (100)	total: 16.5s	remaining: 2m 26s
200:	test: 0.7362088	best: 0.7362088 (200)	total: 33.6s	remaining: 2m 13s
300:	test: 0.7367900	best: 0.7367900 (300)	total: 51.3s	remaining: 1m 59s
400:	test: 0.7371648	best: 0.7371808 (396)	total: 1m 9s	remaining: 1m 44s
500:	test: 0.7373751	best: 0.7373897 (474)	total: 1m 28s	remaining: 1m 27s
600:	test: 0.7374397	best: 0.7374534 (570)	total: 1m 46s	remaining: 1m 10s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7374533679
bestIteration = 570

Shrink model to first 571 iterations.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015370 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you